In [1]:
# importamos las librerias necesarias
import pandas as pd
import json
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Cambiar el directorio de trabajo al directorio donde se encuentran los archivos JSON
%cd ./metadata
# Obtener una lista de archivos en el directorio actual
archivos = os.listdir()

# Crear un DataFrame vacío para almacenar los datos
df_metadata = pd.DataFrame()

# Iterar sobre cada archivo en la lista de archivos
for archivo in archivos:
    # Verificar si el archivo tiene la extensión .json
    if archivo.endswith('.json'):
        # Leer el archivo JSON y cargarlo en un DataFrame
        data = pd.read_json(archivo, lines=True)
        # Concatenar los datos del archivo al DataFrame principal
        df_metadata = pd.concat([df_metadata, data], axis=0)
print("finalizo cargan de datos")

c:\Users\sebas\OneDrive\Escritorio\Proyecto-Grupal-Google-yelp\Sprint 2 Ingenieria de Datos\ETL_google\metadata
finalizo cargan de datos


In [5]:
df_metadata

,0,1,2,3,4,5,6,7,8,9,...,91657,91658,91659,91660,91661,91662,91663,91664,91665,91666
0,"{'name': 'Porter Pharmacy', 'address': 'Porter...","{'name': 'City Textile', 'address': 'City Text...","{'name': 'San Soo Dang', 'address': 'San Soo D...","{'name': 'Nova Fabrics', 'address': 'Nova Fabr...","{'name': 'Nobel Textile Co', 'address': 'Nobel...","{'name': 'Matrix International Textiles', 'add...","{'name': 'Vons Chicken', 'address': 'Vons Chic...","{'name': 'Polished Nail Salon & Spa', 'address...","{'name': 'Sweet Rewards Gluten Free Bakery, LL...","{'name': 'PeepNotes', 'address': None, 'gmap_i...",...,"{'name': 'Elks Lodge', 'address': 'Elks Lodge,...","{'name': 'House Hunters NB Realty', 'address':...","{'name': 'Coleman Tower', 'address': 'Coleman ...","{'name': 'Rye Patch Church', 'address': 'Rye P...","{'name': 'Centrix Corporation.', 'address': No...","{'name': 'Martin Park', 'address': 'Martin Par...","{'name': 'George's Snack Store', 'address': 'G...","{'name': 'E&H Ace Hardware', 'address': 'E&H A...","{'name': 'Impact Recycling Solutions', 'addres...","{'name': 'Giant Eagle Prepared Foods', 'addres..."
0,"{'name': 'Porter Pharmacy', 'address': 'Porter...","{'name': 'Pittsburgh Circus School', 'address'...","{'name': 'Freedom ElecTech', 'address': 'Freed...","{'name': 'Dietz Floral & Gifts', 'address': 'D...","{'name': 'Beech Mountain Community Center', 'a...","{'name': 'Tamburitzans', 'address': 'Tamburitz...","{'name': 'Longwood Bluffs', 'address': 'Longwo...","{'name': 'Davis Controls Corporation', 'addres...","{'name': 'G3 Engineering & Surveying', 'addres...","{'name': 'Estrella Insurance', 'address': 'Est...",...,"{'name': 'Aim All Storage', 'address': 'Aim Al...","{'name': 'Bad Brad's Sports Bar LLC', 'address...","{'name': 'Pioneer Tire & Auto Center', 'addres...","{'name': 'Vitantonio Mfg Co', 'address': 'Vita...","{'name': 'Woodbend Apartments', 'address': 'Wo...","{'name': 'On-Call Process Servers', 'address':...","{'name': 'Marquette County Fairgrounds', 'addr...","{'name': 'Tree Island Wire', 'address': 'Tree ...","{'name': 'Broslavsky & Weinman, LLP', 'address...","{'name': 'Gussied Up', 'address': 'Gussied Up,..."
0,"{'name': 'Fenner Lake', 'address': 'Fenner Lak...","{'name': 'Taylor's Archery', 'address': 'Taylo...","{'name': 'Spirit Halloween', 'address': 'Spiri...","{'name': 'Flirt Entertainment Complex', 'addre...","{'name': 'RS Paint & Body Shop', 'address': 'R...","{'name': 'Templeton & Associates Insurance', '...","{'name': 'Mathnasium', 'address': 'Mathnasium,...","{'name': 'U-Haul Neighborhood Dealer', 'addres...","{'name': 'Soho Chic Beauty Salon Inc', 'addres...","{'name': 'Il Forno', 'address': 'Il Forno, 594...",...,"{'name': 'Hibiscus Business Center', 'address'...","{'name': 'LeBella Nails & Spa', 'address': 'Le...","{'name': 'HoneyCombs Lice Boutique', 'address'...","{'name': 'Athena Greek cafe', 'address': 'Athe...","{'name': 'Cameron Park West Disc Golf Course',...","{'name': 'McGregor Park Public Boat Ramp', 'ad...","{'name': 'Flint River', 'address': 'Flint Rive...","{'name': 'Palm Beach Tan', 'address': 'Palm Be...","{'name': 'Eric's Barber Shop', 'address': 'Eri...","{'name': 'Affordable Storage', 'address': 'Aff..."
0,"{'name': 'Mayhem Mutts', 'address': None, 'gma...","{'name': 'Greystone Apartments', 'address': 'G...","{'name': 'Morning Star Holiness Church', 'addr...","{'name': 'Smoke shop', 'address': 'Smoke shop,...","{'name': 'Midsouth Event Center', 'address': '...","{'name': 'L R Jackson Girls Club', 'address': ...","{'name': 'Boston Baptist Church', 'address': '...","{'name': 'First Baptist Church-Morganton', 'ad...","{'name': 'Music & Arts', 'address': 'Music & A...","{'name': 'First in Smiles Dentistry', 'address...",...,"{'name': 'Charles A. Holmes- Rare Coins', 'add...","{'name': 'Family Dollar', 'address': 'Family D...","{'name': 'Arkadelphia Human Development', 'add...","{'name': 'Fresh Market:Meat & Produce, LLC', '...","{'name': 'Gurdon Housing Authority', 'addr

In [4]:
# Eliminamos columnas irrelevantes
df_metadata.drop(columns=['address', 'state', 'relative_results', 'url','hours','description'], inplace=True)

KeyError: "['address', 'state', 'relative_results', 'url', 'hours', 'description'] not found in axis"

In [ ]:
from pandas import json_normalize 

# Desanidar los datos de la columna "MISC"
df_desanidado = json_normalize(df_metadata['MISC'])

In [ ]:
# Vamos a trabajar con una copia de metadata para crear una posible tabla "service_characteristics"
df_gmap_id = df_metadata.copy()
# Resetear el índice del DataFrame df_gmap_id
df_gmap_id.reset_index(drop=True, inplace=True)

# Combinar los DataFrames desanidados
df_service = pd.concat([df_gmap_id, df_desanidado], axis=1)

In [ ]:
# Eliminamos columna MISC
df_metadata.drop(columns=['MISC'], inplace=True)

In [ ]:
def primer_categoria(lista):
    if lista and len(lista) >= 1:
        return lista[0]
    else:
        return None  

# Aplicar la función a cada fila de la columna 'category'
df_metadata['categoria_1'] = df_metadata['category'].apply(primer_categoria)

In [ ]:
# Eliminamos columna category
df_metadata.drop(columns=['category'], inplace=True)

In [ ]:
# Verificar si hay valores duplicados
df_metadata.duplicated().value_counts()

In [ ]:
df_metadata.drop_duplicates(inplace=True)

In [ ]:
# Verificar si hay valores nulos
df_metadata.isnull().sum()